### Decision Tree From Scratch

#### Import Statements

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random 
from pprint import pprint

#### Load and Prepare Data

In [2]:
df=pd.read_csv('C:/Users/Petrofac/Desktop/Decision Tree From Scratch/iris.csv')

###### Format of the data required
    *Last column of the df must contain the label and it must also be called 'label'
    *There should be no missing value    

In [3]:
df.head()
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [4]:
#Code to drop the columns and rows as required
#Drop row
#df.drop(0, axis=0)
#Drop column
#df.drop("sepal_length",axis=1)

In [5]:
df=df.rename(columns={"species":"label"})
#To ensure no missing value
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
label           150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


### Train-Test-Split

In [6]:
def train_test_split(df,test_size):
    
    if isinstance(test_size,float):
        test_size=round(test_size*len(df))

    indices = df.index.tolist()
    test_indices=random.sample(population=indices,k=test_size)

    test_df=df.loc[test_indices]
    train_df=df.drop(test_indices)
    
    return train_df,test_df

In [7]:
random.seed(0)
train_df,test_df=train_test_split(df,test_size=0.1)

In [8]:
# print(test_df.head())
# print(len(test_df))
# print('\n')

# print(train_df.head())
# print(len(train_df))
# print('\n')

# print(df.head())
# print(len(df))
# print('\n')


### Helper Functions

#### Data purity?

In [9]:
def check_ifpure(data):
    label_clmn=data[:,-1]
    unique_check=np.unique(label_clmn)

    if len(unique_check)== 1:
        return True
    else:
        return False

In [10]:
# check_ifpure(train_df.values)
# check_ifpure(train_df[train_df.petal_width<0.8].values)

### Classify Data

In [11]:
def claasify_data(data):
    label_clmn=data[:,-1]
    unique_classes,unique_count=np.unique(label_clmn,return_counts=True)
    index=unique_count.argmax()
    finalClass=unique_classes[index]
    return finalClass


In [12]:
# claasify_data(train_df[train_df.petal_width >1.2].values)

### Potential Splits

In [13]:
def get_appropriate_splt(data):
    
    app_splits={}
    _,column=data.shape
    for col_index in range(column-1):
        app_splits[col_index]=[]
        values=data[:,col_index]
        unique_val=np.unique(values)

        for indx in range(len(unique_val)):
            if indx != 0:
                presnt_val=unique_val[indx]
                prev_val=unique_val[indx-1]
                app_split=(presnt_val+prev_val)/2

                app_splits[col_index].append(app_split)

            
    return app_splits

In [14]:
# approp_split=get_appropriate_splt(train_df.values)
# approp_split

In [15]:
# sns.lmplot(data=train_df,x="petal_width",y="petal_length",hue="label",fit_reg=False, size=6, aspect=1.5)
# #plt.vlines(x=approp_split[3],ymin=1,ymax=7)
# plt.hlines(y=approp_split[2],xmin=0,xmax=2.5)

### Split Data

In [30]:
def split_data(data, split_clmn, split_val):
    
    split_column_val=data[:,split_clmn]
    
    data_belw=data[split_column_val<=split_val]
    data_above=data[split_column_val>split_val]

    return data_belw, data_above

In [31]:
# data_bleew, data_abeev= split_data(data_abeev, split_col, split_val)
# print(data_bleew,"****",data_abeev)

In [32]:
# plotting_df= pd.DataFrame(data, columns=df.columns)
# sns.lmplot(data=plotting_df,x="petal_width",y="petal_length",fit_reg=False,size=6,aspect=1.5)
# plt.vlines(x=split_val,ymin=1,ymax=7)
# plt.xlim(0,2.6)

### Lowest Overall Entropy?

In [33]:
def cal_entropy(data):
    lbl_col=data[:,-1]
    _,counts=np.unique(lbl_col,return_counts=True)

    pblty=counts/counts.sum()
    entropy = sum(pblty*-np.log2(pblty))
    return entropy

In [34]:
# cal_entropy(data_blw)

In [35]:
def cal_total_entropy(data_blw,data_abv):

    total_len_datapoints=len(data_blw)+len(data_abv)
    prob_data_blw=len(data_blw)/total_len_datapoints
    prob_data_abv=len(data_abv)/total_len_datapoints
    entrpy_data_blw=cal_entropy(data_blw)
    entrpy_data_abv=cal_entropy(data_abv)
    total_entropy=(prob_data_blw*entrpy_data_blw + prob_data_abv*entrpy_data_abv)
    return total_entropy

In [36]:
# cal_total_entropy(data_blw,data_abv)


In [37]:
def find_best_split(data, pot_split):
    
    total_entropy=999
    for clmn_indx in pot_split:
        for val in pot_split[clmn_indx]:
            data_blw,data_abv=split_data(data,split_clmn=clmn_indx,split_val=val)
            entropy=cal_total_entropy(data_blw,data_abv)
            
            if(entropy<=total_entropy):
                total_entropy=entropy
                bst_splt_column=clmn_indx
                bst_splt_value=val        
    
    
    return bst_splt_column,bst_splt_value

In [38]:
# find_best_split(data,get_appropriate_splt(data))

### Decision Tree Algorithm 

In [76]:
def dTree_algorithm(data_frame,count=0,min_instaance_req=2,maximum_depth=5):
    
    #Preparing Data
    if count==0:
        global COL_HEADER
        COL_HEADER=data_frame.columns
        data=data_frame.values
        
    else:
        data=data_frame
        
    #Stopping Condition
    if (check_ifpure(data) or (len(data)<min_instaance_req) or (count == maximum_depth)):
        class_of_data=claasify_data(data)
        return class_of_data
    
    #Recurssion
    else:
        count+=1
        #Getting potential split
        possible_splt=get_appropriate_splt(data)
#         print(possible_splt)
        #Determining split column and value
        splt_colmn,splt_val=find_best_split(data,possible_splt)
#         print(splt_colmn,splt_val)
        #Determine data above and below
        data_blw,data_abv=split_data(data,splt_colmn,splt_val)
#         print(data_blw,"***",data_abv)
        attribute_name=COL_HEADER[splt_colmn]
        condition_quest="{} <= {}".format(attribute_name,splt_val)
        sub_dTree={condition_quest:[]}
        
        
        
        yes_class=dTree_algorithm(data_blw,count,min_instaance_req,maximum_depth)
        no_class=dTree_algorithm(data_abv,count,min_instaance_req,maximum_depth)
        
        if yes_class==no_class:
            sub_dTree=yes_class
            
        else:
            sub_dTree[condition_quest].append(yes_class)
            sub_dTree[condition_quest].append(no_class)
              
    return sub_dTree

In [78]:
tree=dTree_algorithm(train_df,min_instaance_req=1,maximum_depth=3)
pprint(tree)

{'petal_width <= 0.8': ['Iris-setosa',
                        {'petal_width <= 1.65': [{'petal_length <= 4.95': ['Iris-versicolor',
                                                                           'Iris-virginica']},
                                                 'Iris-virginica']}]}


## Classification 
       

#### Classify one example

In [ ]:
def predict_example(instance,dTree):
    
    return 

#### Classify many example

In [88]:
instance =test_df.iloc[0]
instance

sepal_length                5.1
sepal_width                 2.5
petal_length                  3
petal_width                 1.1
label           Iris-versicolor
Name: 98, dtype: object

In [89]:
condition=list(tree.keys())[0]
condition.split()
attribute,operator,val=condition.split()

#Check  condition
if instance[attribute]<= float(val):
    

SyntaxError: invalid syntax (<ipython-input-89-fbc503163aa9>, line 6)

In [87]:
print(attribute,operator,value)

petal_width <= 0.8
